# Extracción datos

In [69]:
# VARIABLES GLOBALES

peajes_dict = {}
rutas_dict = {}

texts_dict = {}

## Parseo del texto

Recibe el texto de la página y lo devuelve como un dict, además guarda un dict global con todas las rutas.

In [152]:
import re

def parse_text(text, origen = "null", destino = "null"):
    res = {}

    # Buscar la ruta
    try:
        ruta = re.search(r'¿Cuántos Peajes hay de \w+ a \w+', text)
        sub_ruta = ruta.group(0).split()
        res["destino"] = sub_ruta[-1]
        res["origen"] = sub_ruta[-3]
    except:
        print("Error al buscar origen y destino")

        if "null" not in origen:
            res["origen"] = origen
        else:
            print("no hay origen proporcionado")

        if "null" not in destino:
            res["destino"] = destino
        else:
            print("no hay destino proporcionado")

    if "null" in origen or "null" in destino:
        origen = res["origen"]
        destino = res["destino"]

    # Buscar la distancia y la duración
    distancia_match = re.search(r'Distancia:\s*([\d.,]+)\s*km', text)
    duracion_match = re.search(r'Duración:\s*(.+)', text)

    try:
        res["distancia"] = float(distancia_match.group(1).replace(",",""))
        res["duracion"] = duracion_match.group(1)
    except:
        print("distancia o duracion erroneos")
        distancia = None
        duracion = None

    # Buscar los peajes
    peaje_match = re.search(r'Peaje\s+(\w+)( *(\w*))', text)
    substring = text
    lim = 15
    valor = 0

    peajes_ruta = []

    while (peaje_match != None and lim > 0):
        # Buscar todos los peajes con sus categorías y tarifas
        substring = substring[peaje_match.span()[1]:]

        valor_match = re.search(r'I\n.*', substring)
        peaje_aux = peaje_match.group(0)[6:]
        peajes_ruta.append(peaje_aux)

        valor_aux = valor_match.group().split()[-1][1:]
        valor += int(valor_aux)

        if peaje_aux not in peajes_dict:
            peajes_dict[peaje_aux] = {
                "valor" : valor_aux,
                "otros" : []
            }
        else:
            peajes_dict[peaje_aux]["otros"].append(valor_aux)

        peaje_match = re.search(r'Peaje\s+(\w+)( *(\w*))+', substring)

        lim -= 1

    res["peajes"] = peajes_ruta

    # Precio
    res["precioPeajes"] = valor

    #print(res)
    rutas_dict[origen+" - "+destino] = res

    #print("rutas", rutas_dict)
    #print("peajes", peajes_dict)

    return res


## Parseo de la página

Extrae el texto de la página.

In [71]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

def html_parse(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

In [72]:
def remove_accents(input_str):
    """
    Remueve las tildes de los caracteres en input_str.
    """
    # Define el mapeo de caracteres con tilde a sus equivalentes sin tilde
    accents_mapping = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
    }

    # Reemplaza los caracteres con tilde por sus equivalentes sin tilde
    for char, replacement in accents_mapping.items():
        input_str = input_str.replace(char, replacement)

    return input_str

## Conexion entre parseo de página y parseo de texto.

Llama la función de parseo de página, recibe el texto y se lo entrega al parseo de texto, muestra los dos dicts correspondientes a las rutas y peajes que se extraen.

In [139]:
def extraerRuta(origen, destino, forzar = False):
    origen_aux = remove_accents(origen.lower())
    destino_aux = remove_accents(destino.lower())

    if (not forzar and ((origen+" - "+destino) in texts_dict or (destino+" - "+origen) in texts_dict)):
        print("Ruta "+origen+" - "+destino+" desde cache.")
        if (not (origen+" - "+destino) in texts_dict):
            texts_dict[origen+" - "+destino] = texts_dict[destino+" - "+origen]
        elif (not (destino+" - "+origen) in texts_dict):
            texts_dict[destino+" - "+origen] = texts_dict[origen+" - "+destino]

        text = texts_dict[origen+" - "+destino]
    else:
        url = "https://www.peajesencolombia.com/peajes/"+origen_aux+"-"+destino_aux
        print(url)

        text = html_parse(url)
        texts_dict[origen+" - "+destino] = text

    try:
        ruta = parse_text(text, origen, destino)
    except:
        if "undefined" in text.lower():
            print("caso undefined")
            rutas_dict[origen+" - "+destino] = "undefined"
        else:
            rutas_dict[origen+" - "+destino] = "error"

        print("error al buscar ruta, se busca el reciproco")
        if ((destino+" - "+origen) in rutas_dict.keys()):
            rutas_dict[origen+" - "+destino] = rutas_dict[destino+" - "+origen]
            ruta = rutas_dict[destino+" - "+origen]
        else:
            ruta = extraerRuta(destino, origen, True)

    print(ruta)
    return ruta

In [153]:
limite = 500

ciudades = ["Palmira", "Pasto", "Tuluá", "Bogotá", "Pereira",	"Armenia",	"Manizales",	"Valledupar",	"Montería",	"Soledad",	"Cartagena",	"Barranquilla",	"Medellín",	"Bucaramanga",	"Cúcuta"]

for origen in ciudades:

    origen_aux = remove_accents(origen.lower())

    for destino in ciudades:
        if limite > 0:
            limite -= 1
            destino_aux = remove_accents(destino.lower())
            if origen_aux == destino_aux:
                continue
            else:
                extraerRuta(origen, destino)
        else:
            break

print("rutas", rutas_dict)
print("peajes", peajes_dict)

Ruta Palmira - Pasto desde cache.
{'destino': 'Pasto', 'origen': 'Palmira', 'distancia': 401.0, 'duracion': '8h 36 min', 'peajes': ['CIAT', 'VILLARICA', 'TUNIA', 'EL BORDO', 'CANO', 'DAZA'], 'precioPeajes': 46000}
Ruta Palmira - Tuluá desde cache.
{'destino': 'Tuluá', 'origen': 'Palmira', 'distancia': 79.2, 'duracion': '1 hour 18 mins', 'peajes': ['CIAT', 'CERRITO', 'BETANIA'], 'precioPeajes': 27200}
Ruta Palmira - Bogotá desde cache.
{'destino': 'Bogotá', 'origen': 'Palmira', 'distancia': 438.0, 'duracion': '8 h 50 min', 'peajes': ['CIAT', 'CERRITO', 'BETANIA', 'LA URIBE', 'COROZAL', 'CAJAMARCA', 'GUALANDAY', 'CHICORAL', 'CHINAUTA', 'CHUSACA'], 'precioPeajes': 97300}
Ruta Palmira - Pereira desde cache.
{'destino': 'Pereira', 'origen': 'Palmira', 'distancia': 194.0, 'duracion': '3 h 3 min', 'peajes': ['CIAT', 'CERRITO', 'BETANIA', 'LA URIBE', 'CERRITOS II'], 'precioPeajes': 49000}
Ruta Palmira - Armenia desde cache.
{'destino': 'Armenia', 'origen': 'Palmira', 'distancia': 165.0, 'durac

# Refinado de datos

## Rutas vacías

Se encontraron errores en 3 rutas:
- "Pasto - Pereira": "error"
- "Pasto - Manizales": "error"
- "Pasto - Valledupar": "error"

In [154]:
# Se hace un rutas ^-1

rutas_dict["Pasto - Pereira"]= rutas_dict["Pereira - Pasto"]
rutas_dict["Pasto - Manizales"]= rutas_dict["Manizales - Pasto"]
rutas_dict["Pasto - Valledupar"]= rutas_dict["Valledupar - Pasto"]


## Formato tiempo a minutos

In [155]:
def tiempo_a_minutos(tiempo):
    dias = 0
    horas = 0
    minutos = 0

    # Se usa regex para extraer dias, horas y minutos
    dias_match = re.search(r'(\d+)\s*d', tiempo)
    horas_match = re.search(r'(\d+)\s*h', tiempo)
    minutos_match = re.search(r'(\d+)\s*min', tiempo)

    if dias_match:
        dias = int(dias_match.group(1))

    if horas_match:
        horas = int(horas_match.group(1))

    if minutos_match:
        minutos = int(minutos_match.group(1))

    # Se convierte todo a minutos
    return (dias * 24 + horas) * 60 + minutos


In [156]:
for a in rutas_dict:
    print(a)
    tiempo = rutas_dict[a]["duracion"]
    if isinstance(tiempo, str):
        tiempo_minutos = tiempo_a_minutos(tiempo)
    else:
        tiempo_minutos = tiempo

    print(tiempo,"->",tiempo_minutos)

    rutas_dict[a]["duracion"] = tiempo_minutos

print(rutas_dict)


Palmira - Pasto
8h 36 min -> 516
Palmira - Tuluá
1 hour 18 mins -> 78
Palmira - Bogotá
8 h 50 min -> 530
Palmira - Pereira
3 h 3 min -> 183
Palmira - Armenia
2 h 39 min -> 159
Palmira - Manizales
4h 13 min -> 253
Palmira - Valledupar
18 h 24 min -> 1104
Palmira - Montería
15 h 34 min -> 934
Palmira - Soledad
20 h 36 min -> 1236
Palmira - Cartagena
19h 46 min -> 1186
Palmira - Barranquilla
20 h 34 min -> 1234
Palmira - Medellín
7 h 53 min -> 473
Palmira - Bucaramanga
13 h 21 min -> 801
Palmira - Cúcuta
18 h 22 min -> 1102
Pasto - Tuluá
9h 1 min -> 541
Pasto - Bogotá
16 h 29 min -> 989
Pasto - Pereira
11h 23 min -> 683
Pereira - Pasto
683 -> 683
Pasto - Armenia
10 hours 28 mins -> 628
Pasto - Manizales
12 h 23 min -> 743
Manizales - Pasto
743 -> 743
Pasto - Valledupar
1 día 3 horas -> 1620
Valledupar - Pasto
1620 -> 1620
Pasto - Montería
23 h 20 min -> 1400
Pasto - Soledad
1 día 4 horas -> 1680
Pasto - Cartagena
1 día 3 horas -> 1620
Pasto - Barranquilla
1 día 4 horas -> 1680
Pasto - Med

## Precio

1. Para la gasolina

Se toma el promedio de **$15.164 COP por galón**.

Extraído de https://elpais.com/america-colombia/2024-01-26/precio-de-la-gasolina-2024-incremento-y-costo-por-galon-en-las-ciudades-de-colombia.html

Se toma como referencia el **Renault Logan** 2023 cuyo consumo en carretera es de **62 km por galón**.

Extraído de https://www.eltiempo.com/economia/finanzas-personales/los-carros-que-menos-gastan-gasolina-en-colombia-en-2023-786128

https://www.wradio.com.co/2023/09/08/lista-de-carros-que-menos-gasolina-consumen-en-colombia/

2. Para el valor de la hora viajada

Se toma el valor de $6500 por hora, teniendo en cuenta que la hora ordinaria está a \$5652 y se tienen que tener en cuenta algunas horas extra y recargos.

Referencia: https://www.portafolio.co/economia/finanzas/salario-minimo-2024-como-quedo-el-pago-de-las-horas-extra-y-los-recargos-laborales-595878

In [157]:
precioGalon = 15164
kilometrosGalon = 62
precioHora = 6500

for a in rutas_dict:
    print(a)

    galonesRuta = rutas_dict[a]["distancia"] / kilometrosGalon
    precioGasolina = round(galonesRuta * precioGalon)
    rutas_dict[a]["precioGasolina"] = precioGasolina

    precioTiempo = (rutas_dict[a]["duracion"] // 60) * precioHora

    precioTotal = precioGasolina + rutas_dict[a]["precioPeajes"] + precioTiempo
    rutas_dict[a]["precioTotal"] = precioTotal

    print(precioTotal)

Palmira - Pasto
196077
Palmira - Tuluá
53071
Palmira - Bogotá
256426
Palmira - Pereira
115949
Palmira - Armenia
100656
Palmira - Manizales
159544
Palmira - Valledupar
538628
Palmira - Montería
432977
Palmira - Soledad
268769
Palmira - Cartagena
264452
Palmira - Barranquilla
272503
Palmira - Medellín
235377
Palmira - Bucaramanga
388725
Palmira - Cúcuta
483329
Pasto - Tuluá
228408
Pasto - Bogotá
431079
Pasto - Pereira
291335
Pereira - Pasto
291335
Pasto - Armenia
276042
Pasto - Manizales
332764
Manizales - Pasto
332764
Pasto - Valledupar
326265
Valledupar - Pasto
326265
Pasto - Montería
278593
Pasto - Soledad
311164
Pasto - Cartagena
304648
Pasto - Barranquilla
314500
Pasto - Medellín
420986
Pasto - Bucaramanga
268880
Pasto - Cúcuta
626337
Tuluá - Bogotá
273994
Tuluá - Pereira
63661
Tuluá - Armenia
68394
Tuluá - Manizales
117479
Tuluá - Valledupar
249452
Tuluá - Montería
404766
Tuluá - Soledad
270050
Tuluá - Cartagena
498831
Tuluá - Barranquilla
273385
Tuluá - Medellín
193067
Tuluá - Buc

# Extracción de matrices

Se generarán matrices para la distancia entre las ciudades, el tiempo que toma ir de una a otra y el precio

In [158]:
matriz_distancia= []
matriz_tiempo= []
matriz_precio= []

# se tiene en cuenta el mismo orden que se usa en 'conexion entre parseo'
# ciudades = ["Palmira", "Pasto", "Tuluá", "Bogotá", "Pereira",	"Armenia",	"Manizales",	"Valledupar",	"Montería",	"Soledad",	"Cartagena",	"Barranquilla",	"Medellín",	"Bucaramanga",	"Cúcuta"]

for i in range(len(ciudades)):
    origen = ciudades[i]

    fila_distancia = []
    fila_tiempo = []
    fila_precio = []

    for j in range(len(ciudades)):
        destino = ciudades[j]

        if i == j:
            fila_distancia.append(0)
            fila_tiempo.append(0)
            fila_precio.append(0)
        else:
            # "Armenia - Barranquilla"
            ruta = rutas_dict[origen+" - "+destino]

            fila_distancia.append(ruta["distancia"])
            fila_tiempo.append(ruta["duracion"])
            fila_precio.append(ruta["precioTotal"])

    matriz_distancia.append(fila_distancia)
    matriz_tiempo.append(fila_tiempo)
    matriz_precio.append(fila_precio)

print(matriz_distancia)
print(matriz_tiempo)
print(matriz_precio)

[[0, 401.0, 79.2, 438.0, 194.0, 165.0, 241.0, 1102.0, 807.0, 1.098, 1.031, 1.237, 401.0, 752.0, 945.0], [401.0, 0, 469.0, 825.0, 584.0, 555.0, 634.0, 1.494, 1.198, 1.489, 1.424, 1.636, 796.0, 1.143, 1331.0], [79.2, 469.0, 0, 361.0, 118.0, 88.7, 170.0, 1.031, 726.0, 1.021, 963.0, 1.165, 329.0, 674.0, 865.0], [438.0, 825.0, 361.0, 0, 330.0, 281.0, 290.0, 867.0, 787.0, 998.0, 1.045, 1.002, 415.0, 429.0, 558.0], [194.0, 584.0, 118.0, 330.0, 0, 46.1, 543.0, 915.0, 614.0, 906.0, 847.0, 1.05, 214.0, 558.0, 750.0], [165.0, 555.0, 88.7, 281.0, 46.1, 0, 98.4, 942.0, 659.0, 1.076, 886.0, 1.077, 258.0, 585.0, 776.0], [241.0, 634.0, 170.0, 290.0, 543.0, 98.4, 0, 865.0, 597.0, 889.0, 831.0, 998.0, 198.0, 507.0, 700.0], [1102.0, 1.494, 1.031, 867.0, 915.0, 942.0, 865.0, 0, 433.0, 298.0, 369.0, 303.0, 749.0, 449.0, 541.0], [807.0, 1.198, 726.0, 787.0, 614.0, 659.0, 597.0, 433.0, 0, 343.0, 254.0, 357.0, 405.0, 616.0, 704.0], [1.098, 1.489, 1.021, 998.0, 906.0, 1.076, 889.0, 298.0, 343.0, 0, 129.0, 16.2